In [61]:
import requests
import json
import hashlib
import time

input = {
    "input": " ",
    "prompt": "问答",
    "question": "知乎上线多长时间了？",
    "<ans>":"",
}

endpoint_name = "cpm-bee-230915061054KERH"
ak = "9ae2487f531411eeb4eb0242ac120004"
sk = "ISzGfWFBFGd0CEdV=-ovFYV/H0H#RkFj"
host = "saas-1222385307.us-west-2.elb.amazonaws.com"

timestemp = str(int(time.time()))
hl = hashlib.md5()
hl.update(f"{timestemp}{sk}".encode(encoding='utf-8'))
sign = hl.hexdigest()

url = f"http://{host}/inference"

payload = {
   "endpoint_name": endpoint_name,
   "input": json.dumps(input),
   "ak": ak,
   "timestamp": timestemp,
   "sign": sign
}

response = requests.post(url=url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
if response.status_code != 200:
    print(response.reason, response.text)
else:
    obj_json = response.json()
    if obj_json['code'] != 0:
        print(obj_json['<ans>'])
    else:
        data_str = obj_json['data']['data']
        data_obj = json.loads(data_str)
        print(data_obj['<ans>'])

知乎于2005年1月21日上线，至今已有16年的时间。在这16年里，知乎不断发展壮大，如今已经成为了一个拥有数百万用户、涵盖多个领域的知名问答社区和知识分享平台。


In [2]:
from langchain import SerpAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.tools.python.tool import PythonAstREPLTool

from typing import Dict, Tuple
import os
import json

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

# 为了使用谷歌搜索（SERPAPI）， WolframAlpha，您需要自行申请它们的 API KEY，然后填入此处
os.environ['SERPAPI_API_KEY'] = '7439b4bb954e332c71776ee046986e08070b6e5db355af2edc1d6072bad17c14'
os.environ['WOLFRAM_ALPHA_APPID'] = '7439b4bb954e332c71776ee046986e08070b6e5db355af2edc1d6072bad17c14'

search = SerpAPIWrapper()
WolframAlpha = WolframAlphaAPIWrapper()
python = PythonAstREPLTool()


def tool_wrapper_for_qwen(tool):
    def tool_(query):
        query = json.loads(query)["query"]
        return tool.run(query)
    return tool_


# 工具描述：
TOOLS = [
    {
        'name_for_human':
            'google search',
        'name_for_model':
            'Search',
        'description_for_model':
            'useful for when you need to answer questions about current events.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "search query of google",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(search)
    },
    {
        'name_for_human':
            'Wolfram Alpha',
        'name_for_model':
            'Math',
        'description_for_model':
            'Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "the problem to solved by Wolfram Alpha",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(WolframAlpha)
    },
    {
        'name_for_human':
            'python',
        'name_for_model':
            'python',
        'description_for_model':
            "A Python shell. Use this to execute python commands. When using this tool, sometimes output is abbreviated - Make sure it does not look abbreviated before using it in your answer. "
            "Don't add comments to your python code.",
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "a valid python command.",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(python)
    }

]


In [4]:
TOOL_DESC = """{name_for_model}: Call this tool to interact with the {name_for_human} API. What is the {name_for_human} API useful for? {description_for_model} Parameters: {parameters} Format the arguments as a JSON object."""

REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tool_descs}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {query}"""


def build_planning_prompt(TOOLS, query):
    tool_descs = []
    tool_names = []
    for info in TOOLS:
        tool_descs.append(
            TOOL_DESC.format(
                name_for_model=info['name_for_model'],
                name_for_human=info['name_for_human'],
                description_for_model=info['description_for_model'],
                parameters=json.dumps(
                    info['parameters'], ensure_ascii=False),
            )
        )
        tool_names.append(info['name_for_model'])
    tool_descs = '\n\n'.join(tool_descs)
    tool_names = ','.join(tool_names)

    prompt = REACT_PROMPT.format(
        tool_descs=tool_descs, tool_names=tool_names, query=query)
    return prompt


prompt_1 = build_planning_prompt(TOOLS[0:1], query="你好,我最近失眠了")
print(prompt_1)


Answer the following questions as best you can. You have access to the following tools:

Search: Call this tool to interact with the google search API. What is the google search API useful for? useful for when you need to answer questions about current events. Parameters: [{"name": "query", "type": "string", "description": "search query of google", "required": true}] Format the arguments as a JSON object.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 你好,我最近失眠了


In [5]:
import requests
import json
import hashlib
import time

input = {
    "input": " ",
    "prompt": "问答",
    "question": prompt_1,
    "<ans>": "",
}

endpoint_name = "cpm-bee-230915061054KERH"
ak = "9ae2487f531411eeb4eb0242ac120004"
sk = "ISzGfWFBFGd0CEdV=-ovFYV/H0H#RkFj"
host = "saas-1222385307.us-west-2.elb.amazonaws.com"

timestemp = str(int(time.time()))
hl = hashlib.md5()
hl.update(f"{timestemp}{sk}".encode(encoding='utf-8'))
sign = hl.hexdigest()

url = f"http://{host}/inference"

payload = {
    "endpoint_name": endpoint_name,
    "input": json.dumps(input),
    "ak": ak,
    "timestamp": timestemp,
    "sign": sign
}

response = requests.post(url=url, data=json.dumps(payload), headers={
                         'Content-Type': 'application/json'})
if response.status_code != 200:
    print(response.reason, response.text)
else:
    obj_json = response.json()
    if obj_json['code'] != 0:
        print(obj_json['<ans>'])
    else:
        data_str = obj_json['data']['data']
        data_obj = json.loads(data_str)
        print(data_obj['<ans>'])
